# R Setting

In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
library(tidyverse)
library(grDevices)
library(showtext)
library(ggforce)
library(ggpubr)
library(sjPlot)
library(extrafont)
library(psych)

font_add_google(
    name='Source Serif 4',
    family='ssp',
    db_cache=FALSE
)

showtext_auto()

THEME_DEFAULT <- theme_bw(
    base_size=10,
    base_family='ssp',
) + theme(
        axis.title.x=element_text(colour='grey20', size=10, face='bold'),
        axis.title.y=element_text(colour='grey20', size=10, face='bold'),
        axis.text.x=element_text(colour='grey20', size=10),
        axis.text.y=element_text(colour='grey20', size=10),
        strip.text.x=element_text(colour='grey20', size=10, face='bold'),
        strip.text.y=element_text(colour='grey20', size=10, face='bold'),
        legend.title=element_text(colour='grey20', size=10, face='bold'),
        legend.text=element_text(colour='grey20', size=10),
        legend.position='top',
        legend.box.spacing= unit(0, 'cm')
    )

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


R[write to console]: Loading required package: sysfonts

R[write to console]: Loading required package: showtextdb

R[write to console]: Learn more about sjPlot with 'browseVignettes("sjPlot")'.

R[write to console]: Registering fonts with R

R[write to console]: 
Attaching package: ‘extrafont’


R[write to console]: The following object is masked from ‘package:showtextdb’:

    font_install


R[write to console]: 
Attaching package: ‘psych’


R[write to console]: The following objects are masked from ‘package:ggplot2’:

    %+%, alpha




# Implementation

## Incentive

In [67]:
import numpy as np
from numpy.typing import NDArray
from abc import abstractmethod, ABC
from typing import Dict, Optional, Callable, Union, Tuple
import copy

class BaseIncentive(ABC):
    def __init__(
            self,
            compensations: NDArray[float],
            random_state: int = None,
            **kwargs
    ) -> None:
        """
        Abstract base class for the incentive strategy

        Parameters
        ----------
        compensations: NDArray[float]
            An array of compensations that will be given to the user for their behavior change.
        random_state:
            Random seed
        """
        self.random_state = random_state
        self._compensations = np.asarray(compensations)
        self._random = np.random.default_rng(random_state)

    @abstractmethod
    def choose(self, context: int = None) -> Tuple[float, Optional[Dict]]:
        """
        It returns the best compensation and optionally gives its corresponding information.

        Parameters
        ----------
        context: int
            A context that needs to be considered to estimate the best compensation

        Returns
        ---
        compensation, information: Tuple[float, Optional[Dict]]
            'compensation' is the best compensation;
            'information' is the optional information explaining why such a compensation is estimated.
        """
        pass

    @abstractmethod
    def update(self, compensation: float, response: bool, context: int = None):
        """
        It updates the compensation estimation.

        Parameters
        ----------
        compensation: float
            The compensation that was suggested to the user.
        response: bool
            Whether the user accepts the compensation and changes his/her behavior.
        context:
            A context that such a compensation and a response are observed.
        """
        pass

    def __str__(self):
        args = ', '.join([f'{k}={v}' for k, v in self.__dict__.items() if not k.startswith('_')])
        cls = self.__class__.__name__
        return f'{cls}({args})'

    def __repr__(self):
        args = ', '.join([f'{k}={v}' for k, v in self.__dict__.items() if not k.startswith('_')])
        cls = self.__class__.__name__
        return f'{cls}({args})'


class StaticIncentive(BaseIncentive):
    def __init__(
            self,
            base_compensation: float,
            **kwargs
    ) -> None:
        """
        It gives the same amount of compensations everytime.

        Parameters
        ----------
        base_compensation: float
            The amount of compensations that will be suggested.
        """
        super().__init__(**kwargs)
        self.base_compensation = base_compensation

    def choose(self, context: int = None) -> Tuple[float, Optional[Dict]]:
        return self.base_compensation, None

    def update(self, compensation: float, response: bool, context: int = None):
        return


class RandomIncentive(BaseIncentive):
    def __init__(
            self,
            **kwargs
    ) -> None:
        """
        It uniformly and randomly chooses the compensation among all possible compensations.
        """
        super().__init__(**kwargs)

    def choose(self, context: int = None) -> Tuple[float, Optional[Dict]]:
        return self._random.choice(self._compensations), {i: (1.0 / len(self._compensations)) for i in self._compensations}

    def update(self, compensation: float, response: bool, context: int = None):
        return


class ThompsonSamplingIncentive(BaseIncentive):
    def __init__(
            self,
            w: float = 1.0,
            decay_factor: float = 1.0,
            **kwargs
    ) -> None:
        """
        It chooses the optimal compensation that maximizes the expected acceptance or adherence rate using Thompson sampling.

        Parameters
        ----------
        w: float, default = 1.0
            The weight that is summed into the alpha or beta parameter of the Beta distribution every update.
            The larger 'w' leads to growing trials, meaning faster exploitation.
        decay_factor: float, default = 1.0
            The weight multiplied to the alpha and beta parameters of the Beta distribution every update.
            This parameter is used to decay the effect of the past trials.
            The decay_factor equals to 1.0 means that there is no decay.
        """
        super().__init__(**kwargs)
        self.w = w
        self.decay_factor = decay_factor
        self._alpha = np.ones(len(self._compensations)) * self.w
        self._beta = np.ones(len(self._compensations)) * self.w

    def choose(self, context: int = None) -> Tuple[float, Optional[Dict]]:
        E_success, info = list(), dict()
        for a, b, c in zip(self._alpha, self._beta, self._compensations):
            e = self._random.beta(a, b)
            E_success.append(e)
            info[f'{c}_alpha'] = a
            info[f'{c}_beta'] = b
            info[f'{c}_success'] = e
        return self._compensations[np.argmax(E_success)].item(0), info

    def update(self, compensation: float, response: bool, context: int = None):
        self._alpha = np.clip(self._alpha * self.decay_factor, a_min=self.w, a_max=None)
        self._beta = np.clip(self._beta * self.decay_factor, a_min=self.w, a_max=None)

        idx = np.flatnonzero(self._compensations == compensation).item(0)
        self._alpha[idx] = self._alpha[idx] + (self.w if response else 0)
        self._beta[idx] = self._beta[idx] + (0 if response else self.w)


class MOThompsonSamplingIncentive(ThompsonSamplingIncentive):
    def __init__(
            self,
            frame: str = 'gain',
            **kwargs
    ) -> None:
        """
        It chooses the optimal compensation that
        - Maximizes the expected acceptance rate
        - Minimizes the expected compensation will be given to the user

        Parameters
        ----------
        frame: str, default = 'gain'
            This parameter indicates how the compensation is framed to the user.
            If it sets to 'gain', the compensation is given to the users when they change their behaviors.
            Otherwise, the penalty is deducted from the individual budget when they decline to change their behaviors.
        """
        super().__init__(**kwargs)
        if frame is None or frame not in ('gain', 'loss'):
            raise ValueError('the argument, "frame", should be one of "gain" or "loss".')
        self.frame = frame

    def choose(self, context: int = None) -> Tuple[float, Optional[Dict]]:
        E_success, E_cost, info = list(), list(), dict()
        for a, b, c in zip(self._alpha, self._beta, self._compensations):
            e_success = self._random.beta(a, b)
            e_cost = e_success * c if self.frame == 'gain' else (1 - e_success) * c
            E_success.append(e_success)
            E_cost.append(e_cost)
            info[f'{c}_alpha'] = a
            info[f'{c}_beta'] = b
            info[f'{c}_success'] = e_success
            info[f'{c}_cost'] = e_cost

        if self.frame == 'gain':
            direction = ('max', 'min')
        else:
            direction = ('max', 'max')
        E = np.column_stack((E_success, E_cost))
        I_opt = self._random.choice(self._find_pareto_frontiers(E, direction))
        compensation = self._compensations[I_opt].item(0)
        return compensation, info

    @classmethod
    def _is_dominated(cls, u: int, v: int, estimates: NDArray, direction: tuple):
        """
        At least one objective is better than another, 
        and other objectives are equal to or better than others.
        """
        n_objectives = estimates.shape[1]

        for i in np.arange(n_objectives):
            if direction[i] == 'max':
                is_dominated = estimates[u, i] > estimates[v, i]
            else:
                is_dominated = estimates[u, i] < estimates[v, i]

            for j in np.arange(n_objectives):
                if i == j:
                    continue
                else:
                    if direction[j] == 'max':
                        is_dominated = is_dominated and estimates[u, j] >= estimates[v, j]
                    else:
                        is_dominated = is_dominated and estimates[u, j] <= estimates[v, j]

            if is_dominated:
                return True

        return False

    @classmethod
    def _is_incomparable(cls, u: int, v: int, estimates: NDArray, direction: tuple):
        """
        At least one objective is better than another, 
        but at least another one objective is worse than others.
        """
        n_objectives = estimates.shape[1]

        for i in np.arange(n_objectives):
            if direction[i] == 'max':
                is_dominate = estimates[u, i] > estimates[v, i]
            else:
                is_dominate = estimates[u, i] < estimates[v, i]

            for j in np.arange(n_objectives):
                if direction[j] == 'max':
                    if i != j and estimates[u, j] < estimates[v, j] and is_dominate:
                        return True
                else:
                    if i != j and estimates[u, j] > estimates[v, j] and is_dominate:
                        return True
        return False

    @classmethod
    def _find_pareto_frontiers(cls, estimates: NDArray, direction: tuple):
        """
        One objective is dominated or incomparable toward all other objectives,
        such objective is the pareto frontier.
        """
        frontiers = []
        n = estimates.shape[0]

        for i in np.arange(n):
            is_pareto_frontier = True

            for j in np.arange(n):
                if i == j:
                    continue
                else:
                    is_dominated = cls._is_dominated(i, j, estimates, direction)
                    is_incomparable = cls._is_incomparable(i, j, estimates, direction)
                    is_pareto_frontier = is_pareto_frontier and (is_dominated or is_incomparable)

            if is_pareto_frontier:
                frontiers.append(i)

        return np.asarray(frontiers)


class ContextualIncentive(BaseIncentive):
    def __init__(
            self,
            incentives: Union[Dict[int, BaseIncentive], Callable[[Optional[int]], BaseIncentive], BaseIncentive],
            **kwargs
    ):
        """
        It differently estimates the compensation across contexts.

        Parameters
        ----------
        incentives
            The dictionary of incentive strategies where the key is a context and the value is the incentive strategy object (i.e., BaseIncentive);
            or, the function that takes the context and return the incentive strategy object.
        """
        super().__init__(**kwargs)
        self.incentives = incentives
        if type(incentives) is Dict:
            self._context_incentives = dict(incentives)
        else:
            self._context_incentives = dict()

    def choose(self, context: int = None) -> Tuple[float, Optional[Dict]]:
        if context not in self._context_incentives:
            if isinstance(self.incentives, Callable):
                self._context_incentives[context] = self.incentives(context)
            elif isinstance(self.incentives, BaseIncentive):
                self._context_incentives[context] = copy.copy(self.incentives)
        return self._context_incentives[context].choose(context)

    def update(self, compensation: float, response: bool, context: int = None):
        self._context_incentives[context].update(compensation, response, context)


## User Behavior

In [176]:
import numpy as np
from abc import abstractmethod, ABC
from typing import List
from numpy.typing import NDArray


class BaseBehavior(ABC):
    @abstractmethod
    def _likelihood(
        self, 
        compensation: float,
        rounds: int = None,
        context: int = None
    ) -> float:
        """
        Returns the likelihood (or probability) of behavior occurrences for a given compensation, round, and context.

        Parameters
        ----------
        compensation: float
            The compensation for the behavior change
        rounds: int, optional
            The number of rounds, interactions, or compensation suggestions
        context: int, optional
            The current context

        Returns
        ---------
        float
            The likelihood.
        """
        pass

    @abstractmethod
    def is_valid(self) -> bool:
        """
        Returns whether the current set of parameters are valid or not.

        Returns
        ---
        bool
            The validity of the current set of parameters.
        """
        pass

    def estimate_likelihood(
            self, compensation: float, rounds: int = None, context: int = None
    ) -> float:
        """
        This is the clipped version of the '_likelihood' function.
        """
        return np.clip(
            self._likelihood(compensation, rounds, context),
            a_min=0.0, a_max=1.0
        )

    def __str__(self):
        args = ', '.join([f'{k}={v}' for k, v in self.__dict__.items() if not k.startswith('_')])
        cls = self.__class__.__name__
        return f'{cls}({args})'

    def __repr__(self):
        args = ', '.join([f'{k}={v}' for k, v in self.__dict__.items() if not k.startswith('_')])
        cls = self.__class__.__name__
        return f'{cls}({args})'


class DecayedBehavior(BaseBehavior, ABC):
    def __init__(
        self, 
        decay_step: int = None,
        decay_factor: float = 1.0,
        decay_likelihood_min: float = 0
    ):
        """
        Parameters
        ----------
        decay_step: int, optional
            The number of unit rounds that a likelihood decays.
            For example, when decay_step = 10, the likelihood is multiplied by 'decay_factor' every ten rounds.
        decay_factor: float, optional
            The factor multiplied to the likelihood, ranging from 0 to 1.
            When decay_factor = 1, the likelihood does not decrease at all.
        decay_likelihood_min: float, optional
            The minimum likelihood
        """
        self.decay_step = decay_step
        self.decay_factor = decay_factor
        self.decay_likelihood_min = decay_likelihood_min
    
    def estimate_likelihood(
        self, compensation: float, rounds: int = None, context: int = None
    ) -> float:
        p = np.clip(
            self._likelihood(compensation, rounds, context),
            a_min=0.0, a_max=1.0
        )
        if self.decay_step and self.decay_factor:
            decay = np.power(self.decay_factor, rounds // self.decay_step)
            p = np.clip(p * decay, a_min=self.decay_likelihood_min, a_max=1.0)
        return p


class StaticBehavior(DecayedBehavior):
    def __init__(
        self,
        likelihood: float = 0,
        **kwargs
    ):
        """
        The behavior occurs at the 'likelihood' with no respect to the compensation.

        Parameters
        ----------
        likelihood: float
            The base likelihood of behavior occurrences

        """
        super().__init__(**kwargs)
        self.likelihood = likelihood

    def _likelihood(self, compensation: float, rounds: int = None, context: int = None) -> float:
        return self.likelihood

    def is_valid(self) -> bool:
        return 0 <= self.likelihood <= 1


class StepBehavior(DecayedBehavior):
    def __init__(
        self, 
        likelihood_0: float,
        likelihood_1: float,
        threshold: float,
        **kwargs
    ):
        """
        The behavior occurs at the 'likelihood_0' if a compensation is less than 'threshold';
        otherwise occurs at the 'likelihood_1'.

        Parameters
        ----------
        likelihood_0: float
            The likelihood of behavior occurrences when a compensation is less than 'threshold'.
        likelihood_1: float
            The likelihood of behavior occurrences when a compensation is equal to or greater than 'threshold'.
        threshold: float
            The threshold of the compensation.

        """
        super().__init__(**kwargs)
        self.likelihood_0 = likelihood_0
        self.likelihood_1 = likelihood_1
        self.threshold = threshold

    def _likelihood(self, compensation: float, rounds: int = None, context: int = None) -> float:
        return self.likelihood_0 if self.threshold >= compensation else self.likelihood_1

    def is_valid(self) -> bool:
        return 0 <= self.likelihood_0 <= 1 and 0 <= self.likelihood_1 <= 1


class RandomBehavior(DecayedBehavior):
    def __init__(
        self, 
        compensations: NDArray[float],
        random_state: int = None,
        **kwargs
    ) -> None:
        """
        The behavior randomly occurs at each compensation.

        Parameters
        ----------
        compensations: NDArray[float]
            The compensation
        random_state
            Random seed
        """
        super().__init__(**kwargs)
        self.compensations = np.asarray(compensations)
        self.random_state = random_state
        self._random = np.random.default_rng(random_state)
        self.likelihoods = self._random.uniform(low=0, high=1, size=len(self.compensations))

    def _likelihood(self, compensation: float, rounds: int = None, context: int = None) -> float:
        return self.likelihoods[np.argmin(np.abs(self.compensations - compensation))].item(0)

    def is_valid(self) -> bool:
        return True


class SigmoidBehavior(DecayedBehavior):
    def __init__(
            self,
            compensation_0: float,
            compensation_1: float,
            likelihood_0: float = 0.0,
            likelihood_1: float = 1.0,
            **kwargs
    ):
        """
        The behavior occurs by following the sigmoid function;
        there is no need that 'compensation_0' is greater than 'compensation_1' and 'likelihood_0' is greater than 'likelihood_1'.

        Because the sigmoid function does not give the exact zero or one value,
        it will give the approximate value of the likelihood:
        - the likelihood_0 * 0.001 at a compensation_0
        - the likelihood_0 * 0.001 + likelihood_1 * 0.999 at a compensation_1

        Parameters
        ----------
        compensation_0: float
            The compensation that the behavior occurs with the likelihood of 'likelihood_0'.
        compensation_1: float
            The compensation that the behavior occurs with the likelihood of 'likelihood_1'.
        likelihood_0: float
            The likelihood that the behavior occurs when the `compensation_0` is given.
        likelihood_1: float
            The likelihood that the behavior occurs when the `compensation_1` is given.
        """
        super().__init__(**kwargs)
        self.compensation_0 = compensation_0
        self.compensation_1 = compensation_1
        self.likelihood_0 = likelihood_0
        self.likelihood_1 = likelihood_1

    def _likelihood(self, compensation: float, rounds: int = None, context: int = None) -> float:
        mi, ma = min(self.compensation_0, self.compensation_1), max(self.compensation_0, self.compensation_1)
        compensation = np.clip(compensation, a_min=mi, a_max=ma)
        l = np.log(999) * (2 * compensation - self.compensation_0 - self.compensation_1) / (self.compensation_1 - self.compensation_0)
        l = 1 / (1 + np.exp(-l))
        return self.likelihood_0 + (self.likelihood_1 - self.likelihood_0) * l

    def is_valid(self) -> bool:
        return True


class LinearBehavior(DecayedBehavior):
    def __init__(
            self,
            compensation_0: float,
            compensation_1: float,
            likelihood_0: float = 0.0,
            likelihood_1: float = 1.0,
            **kwargs
    ):
        """
        The behavior occurs by following the linear function;
        there is no need that 'compensation_0' is greater than 'compensation_1' and 'likelihood_0' is greater than 'likelihood_1'.

        Parameters
        ----------
        compensation_0: float
            The compensation that the behavior occurs with the likelihood of 'likelihood_0'.
        compensation_1: float
            The compensation that the behavior occurs with the likelihood of 'likelihood_1'.
        likelihood_0: float
            The likelihood that the behavior occurs when the `compensation_0` is given.
        likelihood_1: float
            The likelihood that the behavior occurs when the `compensation_1` is given.
        """
        super().__init__(**kwargs)
        self.compensation_0 = compensation_0
        self.compensation_1 = compensation_1
        self.likelihood_0 = likelihood_0
        self.likelihood_1 = likelihood_1

    def _likelihood(self, compensation: float, rounds: int = None, context: int = None) -> float:
        mi, ma = min(self.compensation_0, self.compensation_1), max(self.compensation_0, self.compensation_1)
        compensation = np.clip(compensation, a_min=mi, a_max=ma)
        return self.likelihood_0 + (compensation - self.compensation_0) * (self.likelihood_1 - self.likelihood_0) / (self.compensation_1 - self.compensation_0)

    def is_valid(self) -> bool:
        return True


class ContextDependentBehavior(BaseBehavior):
    def __init__(
        self,
        n_contexts: int,
        behaviors: List[BaseBehavior],
        **kwargs
    ) -> None:
        """
        The different behavior across contexts.

        Parameters
        ----------
        n_contexts: int
            The number of contexts. This should be the same as the length of 'behaviors'.
        behaviors: List[BaseBehavior]
            The list of behaviors whose length is equal to the `n_contexts`.

        """
        super().__init__()
        self.n_contexts = n_contexts
        self.behaviors = behaviors

    def is_valid(self) -> bool:
        return len(self.behaviors) == self.n_contexts

    def _likelihood(self, compensation: float, rounds: int = None, context: int = None) -> float:
        behavior = self.behaviors[context]
        return behavior.estimate_likelihood(compensation, rounds, context)


class DynamicBehavior(BaseBehavior):
    def __init__(
        self,
        rounds: NDArray[int],
        behaviors: List[BaseBehavior],
        **kwargs
    ) -> None:
        """
        The different behavior across round indices.
        Suppose rounds is [5, 10, 15] and the behaviors = [A, B, C, D].
        - Rounds 4 or before: the behavior A
        - Rounds 5 to 9: the behavior B
        - Rounds 10 to 14: the behavior C
        - Rounds 15 or later: the behavior D

        Parameters
        ----------
        rounds: NDArray[int]
            An array of round indices indicating the which behavior is to be considered.
            Its length should be one longer than the length of 'behaviors'.
        behaviors: List[BaseBehavior]
            The list of different behaviors.
        """
        super().__init__()
        self.rounds = np.asarray(rounds)
        self.behaviors = behaviors

    def is_valid(self) -> bool:
        return len(self.rounds) == len(self.behaviors) - 1

    def _likelihood(self, compensation: float, rounds: int = None, context: int = None) -> float:
        i_behavior = np.digitize(rounds, bins=self.rounds)
        behavior = self.behaviors[i_behavior]
        return behavior.estimate_likelihood(compensation, rounds, context)


## Simulation

In [22]:
from dataclasses import dataclass
import numpy as np
from numpy.typing import NDArray
from typing import Dict, Callable
from collections import defaultdict
from typing import ClassVar


@dataclass
class Simulation:
    incentive: BaseIncentive
    behavior: BaseBehavior
    contexts: NDArray[int]
    compensations: NDArray[float]
    responses: NDArray[bool]
    information: Dict[any, NDArray[any]]
    rounds: int
    successes: int
    costs: float
    name: str = None
    random_state: int = None
    prob_contexts: NDArray[float] = None
    _actor: ClassVar = ray.remote(Simulation.simulate)

    @staticmethod
    def simulate(
            incentive: BaseIncentive,
            behavior: BaseBehavior,
            max_rounds: int,
            name: str = None,
            random_state = None,
            prob_contexts: NDArray[float] = None,
            early_stop_success: int = None,
            early_stop_cost: float = None
    ):
        random = np.random.default_rng(random_state)
        contexts, compensations, responses, information = [], [], [], defaultdict(list)
        successes, costs, rounds = 0, 0, 0

        if prob_contexts is not None:
            prob_contexts = np.asarray(prob_contexts) / np.sum(prob_contexts)
            idx_contexts = np.arange(len(prob_contexts))
        else:
            idx_contexts = None

        for _ in range(max_rounds):
            rounds += 1
            context = random.choice(idx_contexts, p=prob_contexts) if prob_contexts is not None else None
            compensation, info = incentive.choose(context=context)
            threshold = behavior.estimate_likelihood(compensation=compensation, rounds=rounds, context=context)
            response = random.uniform() < threshold
            info = info or dict()

            contexts.append(context)
            compensations.append(compensation)
            responses.append(response)
            for k, v in info.items():
                information[k].append(v)

            successes += (1 if response else 0)
            costs += (compensation if response else 0)

            if early_stop_success and early_stop_success <= successes:
                break

            if early_stop_cost and early_stop_cost <= costs:
                break

            incentive.update(compensation=compensation, response=response, context=context)

        return Simulation(
            incentive=incentive,
            behavior=behavior,
            contexts=np.asarray(contexts),
            compensations=np.asarray(compensations),
            responses=np.asarray(responses),
            information={k: np.asarray(v) for k, v in information.items()},
            rounds=rounds,
            successes=successes,
            costs=costs,
            random_state=random_state,
            prob_contexts=prob_contexts,
            name=name
        )

    @staticmethod
    async def simulate_async(**kwargs):
        return Simulation.simulate(**kwargs)

    @staticmethod
    async def simulate_async_remote(**kwargs):
        return await Simulation._actor.remote(**kwargs)

## Utility

### Visualizing Behavior

In [6]:
import altair as alt
import numpy as np
import pandas as pd


def vis_behavior(
        behavior: BaseBehavior, incentive_min: float, incentive_max: float,
        title: str = None, rounds: int = None, context: int = None
):
    x = np.linspace(incentive_min, incentive_max, 1000)
    y = [behavior.estimate_likelihood(xx, rounds, context) for xx in x]
    df = pd.DataFrame(dict(x = x, y = y))
    
    return alt.Chart(df).mark_line().encode(
        x=alt.X('x:Q').title('Compensation'),
        y=alt.Y('y:Q').title('Likelihood').scale(domain=(0, 1)),
    ).properties(title = title or '')


### Multiprocessing with Ray

In [29]:
from contextlib import contextmanager
import ray


@contextmanager
def on_ray(*args, **kwargs):
    try:
        if ray.is_initialized():
            ray.shutdown()
        ray.init(*args, **kwargs)
        yield None
    finally:
        ray.shutdown()

# Simulation

## Common Setting

In [123]:
import numpy as np


COMPENSATIONS = np.asarray([0, 2, 4, 6, 8, 10])
RANDOM_STATE = 42
INCENTIVES = [
    partial(StaticIncentive, base_compensation=5, compensations=COMPENSATIONS, random_state=RANDOM_STATE),
    partial(RandomIncentive, compensations=COMPENSATIONS, random_state=RANDOM_STATE),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=1.0, decay_factor=1.0, frame='gain')),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=1.5, decay_factor=1.0, frame='gain')),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=2.0, decay_factor=1.0, frame='gain')),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=1.0, decay_factor=0.99, frame='gain')),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=1.5, decay_factor=0.99, frame='gain')),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=2.0, decay_factor=0.99, frame='gain')),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=1.0, decay_factor=0.9, frame='gain')),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=1.5, decay_factor=0.9, frame='gain')),
    partial(ContextualIncentive, compensations=COMPENSATIONS, incentives = MOThompsonSamplingIncentive(compensations=COMPENSATIONS, random_state=RANDOM_STATE, w=2.0, decay_factor=0.9, frame='gain')),
]

NAMES_INCENTIVE = {
    'StaticIncentive(random_state=42, base_compensation=5)': 'static',
    'RandomIncentive(random_state=42)': 'random',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=1.0, decay_factor=1.0, frame=gain))': 'mots-1.0,1',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=1.5, decay_factor=1.0, frame=gain))': 'mots-1.5,1',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=2.0, decay_factor=1.0, frame=gain))': 'mots-2.0,1',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=1.0, decay_factor=0.99, frame=gain))': 'mots-1.0,0.99',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=1.5, decay_factor=0.99, frame=gain))': 'mots-1.5,0.99',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=2.0, decay_factor=0.99, frame=gain))': 'mots-2.0,0.99',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=1.0, decay_factor=0.9, frame=gain))': 'mots-1.0,0.9',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=1.5, decay_factor=0.9, frame=gain))': 'mots-1.5,0.9',
    'ContextualIncentive(random_state=None, incentives=MOThompsonSamplingIncentive(random_state=42, w=2.0, decay_factor=0.9, frame=gain))': 'mots-2.0,0.9'
}

## Scenario #1: Static Behavior - A user behavior with no respect to incentive magnitudes, contexts, and time

In [124]:
import numpy as np
from functools import partial


BEHAVIORS = [
    partial(StaticBehavior, likelihood=.25),
    partial(StaticBehavior, likelihood=.50),
    partial(StaticBehavior, likelihood=.75),
    partial(StaticBehavior, likelihood=1.0),
]

NAMES_BEHAVIOR = {
    'StaticBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, likelihood=0.25)': 'static-0.25',
    'StaticBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, likelihood=0.5)': 'static-0.50',
    'StaticBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, likelihood=0.75)': 'static-0.75',
    'StaticBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, likelihood=1.0)': 'static-1.0'
}

In [78]:
import altair as alt


plots = [vis_behavior(b(), min(COMPENSATIONS), max(COMPENSATIONS), title=str(b())) for b in BEHAVIORS]
alt.vconcat(*plots)

alt.VConcatChart(...)

In [84]:
import asyncio
from itertools import product


TASKS = []

with on_ray():
    for incentive, behavior in product(INCENTIVES, BEHAVIORS):
        for i in range(50):
            task = Simulation.simulate_async_remote(
                incentive=incentive(),
                behavior=behavior(),
                max_rounds=5000,
                random_state=RANDOM_STATE + i,
                early_stop_success=500
            )
            TASKS.append(task)
            
    TASKS = await asyncio.gather(*TASKS)

2024-01-24 16:06:40,219	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [95]:
import pandas as pd


DF = []

for s in TASKS:
    DF.append(dict(
        incentive=NAMES_INCENTIVE[str(s.incentive)],
        behavior=NAMES_BEHAVIOR[str(s.behavior)],
        random_state=s.random_state,
        rounds=s.rounds,
        successes=s.successes,
        costs=s.costs,
        cost_per_success=s.costs / s.successes
    ))

DF = pd.DataFrame(DF)
DF.head()

,incentive,behavior,random_state,rounds,successes,costs,cost_per_success
0,static,static-0.25,42,1944,500,2500,5.0
1,static,static-0.25,43,2065,500,2500,5.0
2,static,static-0.25,44,1985,500,2500,5.0
3,static,static-0.25,45,1845,500,2500,5.0
4,static,static-0.25,46,1811,500,2500,5.0


In [127]:
DF.groupby(['behavior', 'incentive'])[['rounds', 'costs', 'cost_per_success']].mean()

rounds    costs  cost_per_success
behavior    incentive                                        
static-0.25 mots-1.0,0.9   1985.98  1540.24           3.08048
            mots-1.0,0.99  1985.98  1501.04           3.00208
            mots-1.0,1     1985.98  1389.80           2.77960
            mots-1.5,0.9   1985.98  1556.40           3.11280
            mots-1.5,0.99  1985.98  1471.32           2.94264
            mots-1.5,1     1985.98  1353.00           2.70600
            mots-2.0,0.9   1985.98  1546.12           3.09224
            mots-2.0,0.99  1985.98  1452.64           2.90528
            mots-2.0,1     1985.98  1384.68           2.76936
            random         1985.98  2489.92           4.97984
            static         1985.98  2500.00           5.00000
static-0.50 mots-1.0,0.9    996.70  1346.76           2.69352
            mots-1.0,0.99   996.70  1406.08           2.81216
            mots-1.0,1      996.70  1319.32           2.63864
            mots-1.5,0.9    996.70  1341.44           2.68288
            mots-1.5,0.99   996.70  1451.40           2.90280
            mots-1.5,1      996.70  1363.08           2.72616
            mots-2.0,0.9    996.70  1287.16           2.57432
            mots-2.0,0.99   996.70  1388.40           2.77680
            mots-2.0,1      996.70  1380.64           2.76128
            random          996.70  2484.12           4.96824
            static          996.70  2500.00           5.00000
static-0.75 mots-1.0,0.9    665.88  1030.28           2.06056
            mots-1.0,0.99   665.88  1389.20           2.77840
            mots-1.0,1      665.88  1364.00           2.72800
            mots-1.5,0.9    665.88   967.80           1.93560
            mots-1.5,0.99   665.88  1343.32           2.68664
            mots-1.5,1      665.88  1139.92           2.27984
            mots-2.0,0.9    665.88   870.92           1.74184
            mots-2.0,0.99   665.88  1231.16           2.46232
            mots-2.0,1      665.88  1219.76           2.43952
            random          665.88  2485.80           4.97160
            static          665.88  2500.00           5.00000
static-1.0  mots-1.0,0.9    500.00   680.00           1.36000
            mots-1.0,0.99   500.00   118.00           0.23600
            mots-1.0,1      500.00   190.00           0.38000
            mots-1.5,0.9    500.00   438.00           0.87600
            mots-1.5,0.99   500.00    48.00           0.09600
            mots-1.5,1      500.00    80.00           0.16000
            mots-2.0,0.9    500.00   216.00           0.43200
            mots-2.0,0.99   500.00     6.00           0.01200
            mots-2.0,1      500.00    42.00           0.08400
            random          500.00  2452.00           4.90400
            static          500.00  2500.00           5.00000

## Scenario #2: Step Behavior 

In [149]:
import numpy as np
from functools import partial


BEHAVIORS = [
    partial(StepBehavior, likelihood_0=0.01, likelihood_1=1.0, threshold=5),
    partial(StepBehavior, likelihood_0=0.01, likelihood_1=0.5, threshold=5),
    partial(StepBehavior, likelihood_0=0.5, likelihood_1=1.0, threshold=5),
]

NAMES_BEHAVIOR = {
    'StepBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, likelihood_0=0.01, likelihood_1=1.0, threshold=5)': 'step-0.0,1.0',
    'StepBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, likelihood_0=0.01, likelihood_1=0.5, threshold=5)': 'step-0.0,0.5',
    'StepBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, likelihood_0=0.5, likelihood_1=1.0, threshold=5)': 'step-0.5,1.0',
}

In [146]:
import altair as alt


plots = [vis_behavior(b(), min(COMPENSATIONS), max(COMPENSATIONS), title=str(b())) for b in BEHAVIORS]
alt.vconcat(*plots)

alt.VConcatChart(...)

In [147]:
import asyncio
from itertools import product


TASKS = []

with on_ray():
    for incentive, behavior in product(INCENTIVES, BEHAVIORS):
        for i in range(50):
            task = Simulation.simulate_async_remote(
                incentive=incentive(),
                behavior=behavior(),
                max_rounds=5000,
                random_state=RANDOM_STATE + i,
                early_stop_success=500
            )
            TASKS.append(task)
            
    TASKS = await asyncio.gather(*TASKS)

2024-01-24 16:35:25,496	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [150]:
import pandas as pd


DF = []

for s in TASKS:
    DF.append(dict(
        incentive=NAMES_INCENTIVE[str(s.incentive)],
        behavior=NAMES_BEHAVIOR[str(s.behavior)],
        random_state=s.random_state,
        rounds=s.rounds,
        successes=s.successes,
        costs=s.costs,
        cost_per_success=s.costs / s.successes
    ))

DF = pd.DataFrame(DF)
DF.head()

,incentive,behavior,random_state,rounds,successes,costs,cost_per_success
0,static,"step-0.0,1.0",42,5000,44,220,5.0
1,static,"step-0.0,1.0",43,5000,54,270,5.0
2,static,"step-0.0,1.0",44,5000,44,220,5.0
3,static,"step-0.0,1.0",45,5000,49,245,5.0
4,static,"step-0.0,1.0",46,5000,56,280,5.0


In [151]:
DF.groupby(['behavior', 'incentive'])[['rounds', 'costs', 'cost_per_success']].mean()

rounds    costs  cost_per_success
behavior     incentive                                        
step-0.0,0.5 mots-1.0,0.9   2833.42  3713.92           7.42784
             mots-1.0,0.99  1922.58  3696.68           7.39336
             mots-1.0,1     2031.76  3555.12           7.11024
             mots-1.5,0.9   2819.12  3714.64           7.42928
             mots-1.5,0.99  1961.96  3693.12           7.38624
             mots-1.5,1     2056.68  3526.28           7.05256
             mots-2.0,0.9   2802.34  3704.96           7.40992
             mots-2.0,0.99  1915.84  3673.32           7.34664
             mots-2.0,1     2051.18  3515.24           7.03048
             random         1958.94  3945.16           7.89032
             static         5000.00   252.40           5.00000
step-0.0,1.0 mots-1.0,0.9   1318.52  3692.68           7.38536
             mots-1.0,0.99  1187.90  3399.56           6.79912
             mots-1.0,1     1256.74  3234.28           6.46856
             mots-1.5,0.9   1300.60  3601.16           7.20232
             mots-1.5,0.99  1236.14  3355.32           6.71064
             mots-1.5,1     1284.14  3156.88           6.31376
             mots-2.0,0.9   1289.08  3525.80           7.05160
             mots-2.0,0.99  1282.78  3283.56           6.56712
             mots-2.0,1     1375.94  3025.00           6.05000
             random          977.34  3958.72           7.91744
             static         5000.00   252.40           5.00000
step-0.5,1.0 mots-1.0,0.9    779.72  1961.52           3.92304
             mots-1.0,0.99   699.72  2213.60           4.42720
             mots-1.0,1      701.18  2149.08           4.29816
             mots-1.5,0.9    784.84  1951.60           3.90320
             mots-1.5,0.99   701.76  2154.44           4.30888
             mots-1.5,1      702.38  2086.68           4.17336
             mots-2.0,0.9    783.04  1910.28           3.82056
             mots-2.0,0.99   706.50  2125.84           4.25168
             mots-2.0,1      705.64  2070.28           4.14056
             random          668.00  2985.88           5.97176
             static          996.70  2500.00           5.00000

## Scenario #3: Sigmoid Behavior 

In [177]:
import numpy as np
from functools import partial


BEHAVIORS = [
    partial(SigmoidBehavior, likelihood_0=0.0, likelihood_1=1.0, compensation_0=0, compensation_1=10),
    partial(SigmoidBehavior, likelihood_0=0.0, likelihood_1=0.5, compensation_0=0, compensation_1=10),
    partial(SigmoidBehavior, likelihood_0=0.5, likelihood_1=1.0, compensation_0=0, compensation_1=10),
]

NAMES_BEHAVIOR = {
    'SigmoidBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, compensation_0=0, compensation_1=10, likelihood_0=0.0, likelihood_1=1.0)': 'sig-0.0,1.0',
    'SigmoidBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, compensation_0=0, compensation_1=10, likelihood_0=0.0, likelihood_1=0.5)': 'sig-0.0,0.5',
    'SigmoidBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, compensation_0=0, compensation_1=10, likelihood_0=0.5, likelihood_1=1.0)': 'sig-0.5,1.0'
}

In [178]:
import altair as alt


plots = [vis_behavior(b(), min(COMPENSATIONS), max(COMPENSATIONS), title=str(b())) for b in BEHAVIORS]
alt.vconcat(*plots)

alt.VConcatChart(...)

In [179]:
import asyncio
from itertools import product


TASKS = []

with on_ray():
    for incentive, behavior in product(INCENTIVES, BEHAVIORS):
        for i in range(50):
            task = Simulation.simulate_async_remote(
                incentive=incentive(),
                behavior=behavior(),
                max_rounds=5000,
                random_state=RANDOM_STATE + i,
                early_stop_success=500
            )
            TASKS.append(task)
            
    TASKS = await asyncio.gather(*TASKS)

2024-01-24 16:58:43,204	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [180]:
import pandas as pd


DF = []

for s in TASKS:
    DF.append(dict(
        incentive=NAMES_INCENTIVE[str(s.incentive)],
        behavior=NAMES_BEHAVIOR[str(s.behavior)],
        random_state=s.random_state,
        rounds=s.rounds,
        successes=s.successes,
        costs=s.costs,
        cost_per_success=s.costs / s.successes
    ))

DF = pd.DataFrame(DF)
DF.head()

,incentive,behavior,random_state,rounds,successes,costs,cost_per_success
0,static,"sig-0.0,1.0",42,994,500,2500,5.0
1,static,"sig-0.0,1.0",43,989,500,2500,5.0
2,static,"sig-0.0,1.0",44,931,500,2500,5.0
3,static,"sig-0.0,1.0",45,999,500,2500,5.0
4,static,"sig-0.0,1.0",46,972,500,2500,5.0


In [181]:
DF.groupby(['behavior', 'incentive'])[['rounds', 'costs', 'cost_per_success']].mean()

rounds    costs  cost_per_success
behavior    incentive                                        
sig-0.0,0.5 mots-1.0,0.9   2991.04  3718.00           7.43600
            mots-1.0,0.99  2031.46  3772.76           7.54552
            mots-1.0,1     2172.90  3678.56           7.35712
            mots-1.5,0.9   2953.50  3708.60           7.41720
            mots-1.5,0.99  2046.72  3764.00           7.52800
            mots-1.5,1     2176.48  3693.56           7.38712
            mots-2.0,0.9   2947.10  3701.88           7.40376
            mots-2.0,0.99  2018.94  3763.32           7.52664
            mots-2.0,1     2168.68  3682.92           7.36584
            random         1997.72  3924.28           7.84856
            static         1985.98  2500.00           5.00000
sig-0.0,1.0 mots-1.0,0.9   1387.16  3693.36           7.38672
            mots-1.0,0.99  1083.98  3579.16           7.15832
            mots-1.0,1     1059.64  3698.12           7.39624
            mots-1.5,0.9   1373.18  3662.12           7.32424
            mots-1.5,0.99  1089.80  3544.64           7.08928
            mots-1.5,1     1064.12  3674.56           7.34912
            mots-2.0,0.9   1369.42  3668.60           7.33720
            mots-2.0,0.99  1120.44  3521.84           7.04368
            mots-2.0,1     1072.80  3689.52           7.37904
            random          990.30  3908.56           7.81712
            static          996.70  2500.00           5.00000
sig-0.5,1.0 mots-1.0,0.9    789.76  1924.40           3.84880
            mots-1.0,0.99   694.20  2483.32           4.96664
            mots-1.0,1      671.52  2708.04           5.41608
            mots-1.5,0.9    790.84  1871.88           3.74376
            mots-1.5,0.99   695.16  2463.92           4.92784
            mots-1.5,1      672.86  2655.24           5.31048
            mots-2.0,0.9    786.92  1914.44           3.82888
            mots-2.0,0.99   699.44  2408.36           4.81672
            mots-2.0,1      676.44  2646.64           5.29328
            random          667.26  2957.68           5.91536
            static          665.88  2500.00           5.00000

## Scenario #4: Linear Behavior 

In [188]:
import numpy as np
from functools import partial


BEHAVIORS = [
    partial(LinearBehavior, likelihood_0=0.0, likelihood_1=1.0, compensation_0=0, compensation_1=10),
    partial(LinearBehavior, likelihood_0=0.0, likelihood_1=0.5, compensation_0=0, compensation_1=10),
    partial(LinearBehavior, likelihood_0=0.5, likelihood_1=1.0, compensation_0=0, compensation_1=10),
]

NAMES_BEHAVIOR = {
    'LinearBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, compensation_0=0, compensation_1=10, likelihood_0=0.0, likelihood_1=1.0)': 'lin-0.0,1.0',
    'LinearBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, compensation_0=0, compensation_1=10, likelihood_0=0.0, likelihood_1=0.5)': 'lin-0.0,0.5',
    'LinearBehavior(decay_step=None, decay_factor=1.0, decay_likelihood_min=0, compensation_0=0, compensation_1=10, likelihood_0=0.5, likelihood_1=1.0)': 'lin-0.5,1.0'
}

In [183]:
import altair as alt


plots = [vis_behavior(b(), min(COMPENSATIONS), max(COMPENSATIONS), title=str(b())) for b in BEHAVIORS]
alt.vconcat(*plots)

alt.VConcatChart(...)

In [184]:
import asyncio
from itertools import product


TASKS = []

with on_ray():
    for incentive, behavior in product(INCENTIVES, BEHAVIORS):
        for i in range(50):
            task = Simulation.simulate_async_remote(
                incentive=incentive(),
                behavior=behavior(),
                max_rounds=5000,
                random_state=RANDOM_STATE + i,
                early_stop_success=500
            )
            TASKS.append(task)
            
    TASKS = await asyncio.gather(*TASKS)

2024-01-24 17:01:32,530	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [189]:
import pandas as pd


DF = []

for s in TASKS:
    DF.append(dict(
        incentive=NAMES_INCENTIVE[str(s.incentive)],
        behavior=NAMES_BEHAVIOR[str(s.behavior)],
        random_state=s.random_state,
        rounds=s.rounds,
        successes=s.successes,
        costs=s.costs,
        cost_per_success=s.costs / s.successes
    ))

DF = pd.DataFrame(DF)
DF.head()

,incentive,behavior,random_state,rounds,successes,costs,cost_per_success
0,static,"lin-0.0,1.0",42,994,500,2500,5.0
1,static,"lin-0.0,1.0",43,989,500,2500,5.0
2,static,"lin-0.0,1.0",44,931,500,2500,5.0
3,static,"lin-0.0,1.0",45,999,500,2500,5.0
4,static,"lin-0.0,1.0",46,972,500,2500,5.0


In [190]:
DF.groupby(['behavior', 'incentive'])[['rounds', 'costs', 'cost_per_success']].mean()

rounds    costs  cost_per_success
behavior    incentive                                        
lin-0.0,0.5 mots-1.0,0.9   2897.20  3322.24           6.64448
            mots-1.0,0.99  2191.44  3585.40           7.17080
            mots-1.0,1     2097.80  3563.28           7.12656
            mots-1.5,0.9   2871.90  3314.72           6.62944
            mots-1.5,0.99  2168.90  3584.32           7.16864
            mots-1.5,1     2120.28  3538.36           7.07672
            mots-2.0,0.9   2890.68  3302.68           6.60536
            mots-2.0,0.99  2150.20  3618.28           7.23656
            mots-2.0,1     2050.48  3640.52           7.28104
            random         1999.88  3678.04           7.35608
            static         1985.98  2500.00           5.00000
lin-0.0,1.0 mots-1.0,0.9   1425.28  3300.08           6.60016
            mots-1.0,0.99  1071.28  3624.80           7.24960
            mots-1.0,1     1011.98  3693.92           7.38784
            mots-1.5,0.9   1412.36  3276.80           6.55360
            mots-1.5,0.99  1027.04  3658.80           7.31760
            mots-1.5,1     1008.66  3674.68           7.34936
            mots-2.0,0.9   1406.82  3274.32           6.54864
            mots-2.0,0.99  1014.10  3670.96           7.34192
            mots-2.0,1     1007.38  3678.32           7.35664
            random          999.62  3678.16           7.35632
            static          996.70  2500.00           5.00000
lin-0.5,1.0 mots-1.0,0.9    783.98  1763.16           3.52632
            mots-1.0,0.99   702.66  2534.60           5.06920
            mots-1.0,1      677.50  2810.96           5.62192
            mots-1.5,0.9    785.66  1752.32           3.50464
            mots-1.5,0.99   699.78  2550.72           5.10144
            mots-1.5,1      676.50  2821.32           5.64264
            mots-2.0,0.9    783.06  1739.52           3.47904
            mots-2.0,0.99   704.04  2533.92           5.06784
            mots-2.0,1      676.80  2827.76           5.65552
            random          667.90  2877.84           5.75568
            static          665.88  2500.00           5.00000

## Scenario #5: Random Behavior

In [198]:
import asyncio
from itertools import product


TASKS = []

with on_ray():
    for i in range(500):
        for incentive in INCENTIVES:
            task = Simulation.simulate_async_remote(
                incentive=incentive(),
                behavior=RandomBehavior(compensations=COMPENSATIONS, random_state=RANDOM_STATE + i),
                max_rounds=5000,
                random_state=RANDOM_STATE + i,
                early_stop_success=500
            )
            TASKS.append(task)
            
    TASKS = await asyncio.gather(*TASKS)

2024-01-24 17:06:46,905	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [199]:
import pandas as pd


DF = []

for s in TASKS:
    DF.append(dict(
        incentive=NAMES_INCENTIVE[str(s.incentive)],
        behavior='random',
        random_state=s.random_state,
        rounds=s.rounds,
        successes=s.successes,
        costs=s.costs,
        cost_per_success=s.costs / s.successes
    ))

DF = pd.DataFrame(DF)
DF.head()

,incentive,behavior,random_state,rounds,successes,costs,cost_per_success
0,static,random,42,567,500,2500,5.0
1,static,random,43,5000,118,590,5.0
2,static,random,44,1137,500,2500,5.0
3,static,random,45,673,500,2500,5.0
4,static,random,46,1687,500,2500,5.0


In [200]:
DF.groupby(['incentive'])[['rounds', 'costs', 'cost_per_success']].mean()

,rounds,costs,cost_per_success
incentive,,,
"mots-1.0,0.9",949.110,1285.940,2.571880
"mots-1.0,0.99",817.280,1534.336,3.068672
"mots-1.0,1",792.456,1481.128,2.962256
"mots-1.5,0.9",938.492,1234.596,2.469192
"mots-1.5,0.99",811.154,1497.500,2.995000
"mots-1.5,1",789.560,1490.724,2.981448
"mots-2.0,0.9",926.980,1174.684,2.349368
"mots-2.0,0.99",806.902,1485.368,2.970736
"mots-2.0,1",788.372,1472.440,2.944880


## Scenario #6: Context-Dependet Behavior

For the ease of simulation, we suppose:
- The user behavior follows sigmoid behavior, because it is more rationale than that the behavior might randomly occur with no respect to the compensation.
- likelihood_0 samples from [0.0, 0.5)
- likelihood_1 samples from [0.5, 1.0)
- The number of contexts are 3 (i.e., WORK, NON-WORK, WEEKEND)
- The number of missions per week is 7 (days) * 17 (missions/day; 9 to 25) = 119
- The probability distributions of context switching are:
  - WORK: 5 (weekdays) * 9 (missions; 9 to 17) / 119 = 0.378
  - NON-WORK: 5 (weekdays) * 8 (missions; 18 to 25) / 119 = 0.336
  - WEEKEND: 2 (weekend) * 17 (missions) / 119 = 0.285

In [206]:
import asyncio
from itertools import product
import numpy as np


TASKS = []

with on_ray():
    for i in range(500):
        gen = np.random.default_rng(RANDOM_STATE + i)
        sigs = [
            SigmoidBehavior(
                likelihood_0=gen.uniform(0, .5), 
                likelihood_1=gen.uniform(.5, 1.0), 
                compensation_0=0, 
                compensation_1=10
            )
            for _ in range(3)
        ]
        behavior = ContextDependentBehavior(
            n_contexts=3,
            behaviors=sigs
        )
        for incentive in INCENTIVES:
            task = Simulation.simulate_async_remote(
                incentive=incentive(),
                behavior=behavior,
                max_rounds=5000,
                random_state=RANDOM_STATE + i,
                early_stop_success=500,
                prob_contexts=np.array([.378, .336, .285])
            )
            TASKS.append(task)
            
    TASKS = await asyncio.gather(*TASKS)

2024-01-24 17:31:02,511	INFO worker.py:1715 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


In [207]:
import pandas as pd


DF = []

for s in TASKS:
    DF.append(dict(
        incentive=NAMES_INCENTIVE[str(s.incentive)],
        random_state=s.random_state,
        rounds=s.rounds,
        successes=s.successes,
        costs=s.costs,
        cost_per_success=s.costs / s.successes
    ))

DF = pd.DataFrame(DF)
DF.head()

,incentive,random_state,rounds,successes,costs,cost_per_success
0,static,42,882,500,2500,5.000
1,random,42,915,500,3230,6.460
2,"mots-1.0,1",42,910,500,2746,5.492
3,"mots-1.5,1",42,898,500,2914,5.828
4,"mots-2.0,1",42,970,500,2610,5.220


In [208]:
DF.groupby(['incentive'])[['rounds', 'costs', 'cost_per_success']].mean()

,rounds,costs,cost_per_success
incentive,,,
"mots-1.0,0.9",1231.110,2381.304,4.762608
"mots-1.0,0.99",1043.310,2846.064,5.692128
"mots-1.0,1",1033.252,2879.568,5.759136
"mots-1.5,0.9",1229.430,2381.328,4.762656
"mots-1.5,0.99",1037.316,2855.492,5.710984
"mots-1.5,1",1027.272,2897.936,5.795872
"mots-2.0,0.9",1227.616,2374.256,4.748512
"mots-2.0,0.99",1040.372,2846.136,5.692272
"mots-2.0,1",1031.402,2879.544,5.759088


## Summary

1. The proposed incentive estimation is almost always better than the random incentive.
2. From the scenario #1 - #4, the cost effectiveness is improved when the user is highly likely to change their behavior even with the smaller compensation.
3. In the scenario #5 (random behavior), our incentive estimation is the best, meaning that it can well learn the user behavior patterns.
4. In the scenario #6 (context-dependent behavior), our incentive estimation is better than the random incentive and some settings showed the better result than the static incentive.
5. The best parameter of the incentive estimation might be:
   - w = 1.5 or larger
   - decay_factor = 0.9
